#Import Libraries

In [ ]:
import tensorflow as tf

print(tf.__version__)

2.18.0


#Create Tensors from constants

In [ ]:
a = tf.constant([1.0, 2.3, 3.14159], dtype=tf.float32);
b = ????

print(a)
print(a.shape)
print(a.dtype)

print(b)

#Operators with Tensors

In [ ]:
tf.reduce_sum(a)

In [ ]:
a+b

In [ ]:
c= tf.add(a,b)
c

In [ ]:
a * b

In [ ]:
a = tf.reshape(a, [1,3])
a

In [ ]:
at = tf.transpose(a)
at

In [ ]:
b = tf.reshape(b, [1,3])

In [ ]:
dot_b_at = b @ at
dot_b_at

In [ ]:
x = tf.constant([[[ -1,  -2,  -3],
...                   [ -4,  -5, -6]],
...                  [[ 7,  8,  9],
...                   [10, 11, 12]]], dtype=tf.float32)
x

In [ ]:
tf.transpose(x, perm=[0, 2, 1])

In [ ]:
y = tf.nn.sigmoid(x)
y

In [ ]:
tf.nn.relu(x)

In [ ]:
tf.reduce_sum(y, axis=[2])

#Variables

In [ ]:
var = tf.Variable([0,0,0], dtype=tf.float32)
var.assign([1,2,3])
var.assign_sub([0.1, 0.2, 0.3])
print(var)

#Creating tf.function
This creates a graph for each function

In [ ]:
w1 = tf.Variable(tf.random.uniform(shape=[1,10],dtype=tf.float32))

In [ ]:
@tf.function(input_signature=[tf.TensorSpec(shape=(1,10), dtype=tf.float32)])
def my_inference(x):
  print("Entering to Graph");
  print(x)
  return tf.reduce_sum(x*w1)

In [ ]:
input = tf.constant([[1,2,3,4,5,6,7,8,9,0]], dtype=tf.float32)
out = my_inference(input)
out

First time log was printed... check the second execution.... log was not printed

In [ ]:
out = my_inference(input)
out

#Automatic Differentiation for model y = Ax

In [ ]:
A = tf.Variable(tf.random.uniform(shape=[3,2],dtype=tf.float32))
A

In [ ]:
@tf.function(input_signature=[tf.TensorSpec(shape=(2,1), dtype=tf.float32)])
def my_linear_function(x):
  return ??

In [ ]:
x_input = tf.Variable([[10],[20]], shape=(2,1), dtype=tf.float32)

In [ ]:
out = my_linear_function(x_input)
out

In [ ]:
with tf.GradientTape() as tape:
  f = my_linear_function(x_input)

g_x = tape.gradient(f, x_input)  # g(x) = df/x = A


#Does Gradient shape make sense?

In [ ]:
g_x

In [ ]:
A

#Solve Simple Linear Regression with Algebra
# Ax = y

Let's create random data:

In [ ]:
from matplotlib import pyplot as plt

t = tf.linspace(-5,5,201)
t = tf.cast(t, dtype=tf.float32)

def linear(x_axis, slope, intersection):
  y = ??
  return y

y = linear(t, 5, 5) + tf.random.normal(shape=[201], stddev=5.0)
plt.plot(t.numpy(), linear(t,5,5))
plt.plot(t.numpy(), y.numpy(), '.')

In [ ]:
ones = tf.ones(shape=[201])
t_t = tf.reshape(t, [201,1])
y_t = tf.reshape(y, [201,1])
ones = tf.reshape(ones, [201,1])
A = tf.concat([t_t, ones], 1);

Solution using Algebra:

$Ax = y$

$A^tAx = A^ty$

$(A^tA)^{-1} A^tAx = (A^t A)^{-1} A^t y$

$x =  (A^tA)^{-1} A^t y$


In [ ]:
sol = ??

Our solution using algebra is:

In [ ]:
sol

#Creating Linear model using tf.Module

In [ ]:
class LinearModel(tf.Module):
  #def __init__(self):
  def __init__(self, name=None):
    super().__init__(name=name)
    initial_w = tf.random.uniform(shape=[??], minval=-1.0, maxval=1.0)
    self.x1 = tf.Variable(initial_w[1])
    self.x0 = tf.Variable(initial_w[0])

  @tf.function(input_signature=[tf.TensorSpec(shape=[None], dtype=tf.float32)])
  def __call__(self, x):
    return ??


#Watch graph of our model

In [ ]:
%load_ext tensorboard
from datetime import datetime
# Set up logging.
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = "logs/func/%s" % stamp
writer = tf.summary.create_file_writer(logdir)

# Create a new model to get a fresh trace
# Otherwise the summary will not see the graph.
new_model = LinearModel()

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True)
tf.profiler.experimental.start(logdir)
# Call only one tf.function when tracing.
z = print(new_model(tf.constant([2.0], dtype=tf.float32)))
with writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=logdir)
%tensorboard --logdir logs/func

Let's use untrained model

In [ ]:
linear_model = LinearModel()

predictions = linear_model(t)

plt.plot(t, predictions, label= 'Predictions')
plt.plot(t, linear(t, 5,5), label='Truth' )
plt.legend()

#Training our model

$MSE = \frac{1}{m}\sum_{i=1}^{m}(\hat{y}_i -y_i)^2$

In [ ]:
def mse(pred, y):
  return ???

Let's use chunks of data (batches/mini-batches)

In [ ]:
batch_size = 16
dataset = tf.data.Dataset.from_tensor_slices((t, y))
dataset = dataset.shuffle(201).batch(batch_size)

Lest create a loop to:


*   Infer on each batch
*   Save gradients
*   Apply gradients to internal variables





In [ ]:
epochs = 50

for epoch in range(epochs):
  for x_batch, y_batch in dataset:
    with tf.GradientTape() as tape:
      batch_loss = mse(???)

    # Update parameters with respect to the gradient calculations
    grads = tape.gradient(batch_loss, linear_model.variables)
    for g,v in zip(grads, linear_model.variables):
        v.assign_sub(0.0001*g)
  # Keep track of model loss per epoch
  loss = mse(linear_model(t), y)
  print(f"Epoch {epoch} Loss  is {loss.numpy()}",)


Let's print our internal variables

In [ ]:
print(linear_model.variables)

#Let's save model and transform to tf-lite model

In [ ]:
tf.saved_model.save(linear_model, "/tmp/linear/")
converter = tf.lite.TFLiteConverter.from_saved_model("/tmp/linear")

tflite_model = converter.convert()

with open('linear_model.tflite', 'wb') as f:
  f.write(tflite_model)